In [31]:
import gspread
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import datetime

In [32]:
s = requests.Session()
headerbse ={
'scheme':'https',
'Accept':'application/json, text/plain, */*',
'Accept-Language':'en-US,en;q=0.8',
'Referer':'https://www.bseindia.com/',
'Sec-Ch-Ua':'"Not_A Brand";v="8", "Chromium";v="120", "Brave";v="120"',
'Sec-Ch-Ua-Mobile': '?0',
'Sec-Ch-Ua-Platform':'"Windows"',
'Sec-Fetch-Mode':'cors',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

In [33]:
DateFrom = (datetime.datetime.now() - datetime.timedelta(days = 7)).strftime('%Y%m%d')
DateTo = datetime.datetime.now().strftime('%Y%m%d')
#Date_From = 20240101
#Kab Tak
#Date_To = 20240116

In [34]:
url = f'https://api.bseindia.com/BseIndiaAPI/api/AnnSubCategoryGetData/w?pageno=1&strCat=New+Listing&strPrevDate={DateFrom}&strScrip=&strSearch=P&strToDate={DateTo}&strType=C&subcategory=-1'
page = s.get(url,headers= headerbse)

In [35]:
Data = page.json()['Table']

In [36]:
if page.status_code == 200:
    gc = gspread.service_account(filename='creds.json')
    spreadsheet_name = 'Notifications and Listings'
    sheet_number = 0 
    sheet_name = 'BSE_Listing'
    sh = gc.open(spreadsheet_name).worksheet(sheet_name)
else :
    print(page.status_code)

In [37]:
if page.status_code == 200:
    existing_data_range = sh.range('A2:L' + str(sh.row_count))
    for cell in existing_data_range:
        cell.value = ''

    sh.update_cells(existing_data_range)
else :
    pass

In [38]:
if page.status_code == 200:
    df = pd.DataFrame(Data)
    values_lists = []
    df_dict = df.to_dict(orient='records')
    for data in df_dict:
        values_lists.append([str(data['NEWS_DT']),str(data['SCRIP_CD']), str(data['SLONGNAME']), str(data['NSURL']),str(data['SUBCATNAME'])])
    sh.append_row([f'Batch ran at: {datetime.datetime.now()}'])
    sh.append_rows(values_lists)
else:
    pass

In [39]:
df.head(10)

,NEWSID,SCRIP_CD,XML_NAME,NEWSSUB,DT_TM,NEWS_DT,CRITICALNEWS,ANNOUNCEMENT_TYPE,QUARTER_ID,FILESTATUS,...,NSURL,SLONGNAME,AGENDA_ID,TotalPageCnt,News_submission_dt,DissemDT,TimeDiff,Fld_Attachsize,SUBCATNAME,AUDIO_VIDEO_FILE
0,0e95cb35-2bc9-4860-b833-a498cc88a577,544108,ANN_544108_0E95CB35-2BC9-4860-B833-A498CC88A577,Gabriel Pet Straps Ltd - 544108 - Listing of e...,2024-02-07T07:29:03.62,2024-02-07T07:29:03.62,1,A,None,N,...,https://www.bseindia.com/stock-share-price/gab...,Gabriel Pet Straps Ltd,45,1,None,2024-02-07T07:29:03.62,None,None,New Listing,None
1,16536fb4-814a-453e-bd59-ca3561ccb05c,544107,ANN_544107_16536FB4-814A-453E-BD59-CA3561CCB05C,BLS E-Services Ltd - 544107 - Listing of equit...,2024-02-06T08:01:12.403,2024-02-06T08:01:12.403,1,A,None,N,...,https://www.bseindia.com/stock-share-price/bls...,BLS E-Services Ltd,45,1,None,2024-02-06T08:01:12.403,None,None,New Listing,None
2,b4a8cfc6-544f-46a3-a2c4-69bb1da092c2,544106,ANN_544106_B4A8CFC6-544F-46A3-A2C4-69BB1DA092C2,Mayank Cattle Food Ltd - 544106 - Listing of e...,2024-02-05T08:29:02.183,2024-02-05T08:29:02.183,1,A,None,N,...,https://www.bseindia.com/stock-share-price/may...,Mayank Cattle Food Ltd,45,1,None,2024-02-05T08:29:02.183,None,None,New Listing,None
3,5120b6e5-1850-468a-b784-cb4884f050be,544105,ANN_544105_5120B6E5-1850-468A-B784-CB4884F050BE,Harshdeep Hortico Ltd - 544105 - Listing of eq...,2024-02-05T08:28:44.47,2024-02-05T08:28:44.47,1,A,None,N,...,https://www.bseindia.com/stock-share-price/har...,Harshdeep Hortico Ltd,45,1,None,2024-02-05T08:28:44.47,None,None,New Listing,None
4,b2ee479e-ce5e-4b35-b156-b03bfbc1faa8,544101,ANN_544101_B2EE479E-CE5E-4B35-B156-B03BFBC1FAA8,Brisk Technovision Ltd - 544101 - Listing of e...,2024-01-31T07:44:56.14,2024-01-31T07:44:56.14,1,A,None,N,...,https://www.bseindia.com/stock-share-price/bri...,Brisk Technovision Ltd,45,1,None,2024-01-31T07:44:56.14,None,None,New Listing,None
5,26de7f01-133e-477c-bcfe-42669c965ea9,544100,ANN_544100_26DE7F01-133E-477C-BCFE-42669C965EA9,Nova Agritech Ltd - 544100 - Listing of equity...,2024-01-31T07:44:38.357,2024-01-31T07:44:38.357,1,A,None,N,...,https://www.bseindia.com/stock-share-price/nov...,Nova Agritech Ltd,45,1,None,2024-01-31T07:44:38.357,None,None,New Listing,None
